## Deep learning prototype model
Tensorflow implementation of a fully connected deep net for arbitrarily shaped input and output layers, with the following features:
- Easy control of net topology and hyperparameters
- Abstracted functions for retrieving/constructing training data
- Exponential decay of learning rate
- Dropout regularisation
- Batch normalisation
- Automatic logging of bulk statistics of each layer through tensorboard (and convenient functions to attach logging to custom functions)


## Visualising progress
To inspect the training progress, run

>  `>tensorboard --logdir=<log_dir>`


from the terminal, where `log_dir` as as defined above

## Requirements
- TensorFlow 1.8
- numpy
- subprocess


In [4]:
import tensorflow as tf
import numpy as np
from subprocess import call
import time, math
import h5py

In [134]:
# Setting dtype for tensorflow and numpy environments
DTYPE = tf.float32
DTYPE_np = np.float32
log_dir = '/tmp/funclearn' # Directory where we dump tensorboard log files

# Useful function for resetting logdir and tensorflow graph
def reset_tf():
    call(["rm","-rf",log_dir+'/'])
    tf.reset_default_graph()
    
    
# Define some functions for initialising layers with appropriate statistics    
def get_weights(shape,dtype):
    # Returns trainable weight variable, initialised from truncated (+\- 2std. dev. only) standard normal distribution
    return tf.Variable(tf.truncated_normal(shape, dtype=dtype),name='weights')

def get_biases(shape,dtype):
    # Returns trainable bias variable, initialised arbitrarily as a small constant
    return tf.Variable(tf.constant(0.01, shape=shape, dtype=dtype),name='biases')

def get_bn_offset(shape,dtype):
    # Returns trainable bias/offset variable for batch normalisation
    return tf.Variable(tf.truncated_normal(shape=shape, dtype=dtype),name='beta_offset')

def get_bn_scale(shape,dtype):
    # Returns trainable scale variable for batch normalisation
    return tf.Variable(tf.constant(1.0, shape=shape, dtype=dtype),name='gamma_scale')


def variable_summaries(var):
    # Attaches mean, stddev, max, min, and a histogram of an input var to a tensor
    # Useful for TensorBoard visualisation
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var,name='mean')
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)),name='stddev')
        
        tf.summary.scalar('mean',mean)
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('max',tf.reduce_max(var))
        tf.summary.scalar('min',tf.reduce_min(var))
        tf.summary.histogram('histogram', var)
        
def batchnorm(logits, is_test, offset, scale, iteration):
    # Get summary statistics of this batch
    mean, variance    = tf.nn.moments(logits, [0],name='moments')
    
    # We'll use an exponential moving average over the training iterations during test time
    # This is a tool to do that
    exp_moving_avg    = tf.train.ExponentialMovingAverage(0.9999, iteration)
    update_moving_avg = exp_moving_avg.apply([mean, variance])
    
    # If this is the test, we use the m,v values we obtained from the exponential moving average 
    # over mean, variance that we obtained from training. otherwise use the batch mean, variance
    mean_cond        = tf.cond(is_test, lambda: exp_moving_avg.average(mean), lambda: mean, name='mean_cond')
    variance_cond    = tf.cond(is_test, lambda: exp_moving_avg.average(variance), lambda: variance, name='variance_cond')
    
    # This applies the following normalisation: x-> scale*(x-mean(x))/(variance_epsilon+std(x)) + offset
    logits_bn = tf.nn.batch_normalization(logits, mean_cond, variance_cond, offset, scale, variance_epsilon=1e-5,name='logits_batchnormed')
    
    return logits_bn, update_moving_avg

def get_layer_complete(input_tensor,input_dim, output_dim, layer_name, is_test, prob_keep, global_step, act_func=tf.nn.relu):
    with tf.name_scope(layer_name):
        with tf.name_scope('weights'):
            weights = get_weights([input_dim, output_dim],DTYPE)
            variable_summaries(weights)
        with tf.name_scope('biases'):
            biases = get_biases([output_dim],DTYPE)
            variable_summaries(biases)
        with tf.name_scope('batchnorm'):
            offset = get_bn_offset([output_dim], DTYPE)
            scale  = get_bn_scale([output_dim], DTYPE)
        
        logits = tf.add(tf.matmul(input_tensor, weights),biases,name='logits')
        tf.summary.histogram('logits', logits)
        logits_bn, update_moving_avg = batchnorm(logits, is_test, offset, scale, global_step)
        tf.summary.histogram('logits_batchNormed', logits_bn)
        activated = act_func(logits_bn, name='activation')
        dropped_out = tf.nn.dropout(activated,prob_keep,name='dropout')
        tf.summary.histogram('activations', activated)
        return dropped_out, update_moving_avg 
    
def func_deep_learner_complete(x,m,n,h,is_test,global_step,prob_keep,act_func=tf.nn.relu,num_layers=5):
    # x is the input variable, probably a TensorFlow placeholder object
    # m is the input dimension
    # n is the output dimension
    # h is the number of hidden neurons
    # act_func is the activation function to be applied
    # is_test is a flag is use to control batch normalisation behaviour during inferene on test data
    # iteration is the iteration counter used inside the training loop; required for batch normalisation
    
    bn_moving_avg_updates = []

    with tf.variable_scope('func_learner'):
        # 0th hidden layer
        hidden_layer, update_moving_avg = get_layer_complete(x,m,h,'hidden_layer_0',is_test,prob_keep,global_step)
        bn_moving_avg_updates.append(update_moving_avg)

        # Other hidden layers
        for i in range(num_layers-1):
            hidden_layer, update_moving_avg = get_layer_complete(hidden_layer,h,h,'hidden_layer_'+str(i+1),is_test,prob_keep,global_step)
            bn_moving_avg_updates.append(update_moving_avg)

        # Output layer
        weights1= tf.get_variable(name='output_layer_weights',
                              shape=[h,n],
                              initializer=tf.random_normal_initializer(),
                              dtype=DTYPE)

    return tf.matmul(hidden_layer,weights1), bn_moving_avg_updates

def train_RBM_complete(input_size=1,output_size=1,hidden_size=1,num_layers=5,model_func=None,unknown_func=None,
                  test_x=None, test_y = None, generate_training_samples_fun=None,
                  batch_size=25, max_steps = 1000, epochs =5,
                  initial_learning_rate = 0.02, decay_rate = 1/math.e, decay_steps=1000,
                  prob_keep = 0.8):
    
    with tf.Session() as sess:
            
        with tf.name_scope('input'):
            # Placeholder variables which will be fed data at train time
            x  = tf.placeholder(DTYPE,[None,input_size],name='x_input')
            y  = tf.placeholder(DTYPE,[None,output_size],name='y_input')
        
        with tf.name_scope('control_inputs'):
            global_step = tf.Variable(0, trainable=False,name='global_step')
            epoch_ind = tf.Variable(0,trainable=False,name='epoch_ind')
            increment_epoch_ind = tf.assign_add(epoch_ind,1,name='increment_epoch_ind')
            is_test = tf.contrib.eager.Variable(False, trainable=False,name='is_test')

        # Predictions of the NN
        y_pred, bn_moving_avg_updates = func_deep_learner_complete(x,input_size,output_size,hidden_size,is_test,global_step,1,num_layers=num_layers)

        # Should add dropout!

        with tf.name_scope('mean_squared_error'):
            mse = tf.losses.mean_squared_error(y,y_pred)
        tf.summary.scalar('mean_squared_error',mse)

        with tf.name_scope('train'):
            global_step_in_epoch = global_step - epoch_ind*max_steps
            learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step_in_epoch, decay_steps, decay_rate, staircase=True)
            tf.summary.scalar('learning_rate', learning_rate)
            train_step = tf.train.AdamOptimizer(learning_rate).minimize(mse, global_step = global_step)

        # Create merged summary object and file writers
        merged_summaries = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter(log_dir + '/train', sess.graph)
        test_writer = tf.summary.FileWriter(log_dir + '/test', sess.graph)
        
        # Initialise variables
        sess.run(tf.global_variables_initializer())
        
        # Train the model
        start_time = time.time()
        old_time = start_time
        for j in range(epochs):
            for i in range(max_steps):
                if i % 100 ==0:
                    # Check how our model performs against the test data
                    [summary, mse_val] = sess.run([merged_summaries,mse], feed_dict={x: test_x, y: test_y, is_test: True})
                    test_writer.add_summary(summary,i + j*max_steps)
                    curr_time = time.time()
                    print('Epoch %d, Step %04d, MSE: %4.3e,  LearnRate: %4.3e, Time taken : %4.3fs' % (sess.run(epoch_ind),i, mse_val, sess.run(learning_rate),curr_time-old_time))
                    old_time = curr_time
                else:
                    # Generate new sample data and train our model
                    train_x, train_y = generate_training_samples_fun(epoch_ind, global_step)
                    summary, _ = sess.run([merged_summaries, train_step], feed_dict={x: train_x, y: train_y, is_test: False})
                    sess.run(bn_moving_avg_updates, feed_dict={x: train_x, y: train_y, is_test: False})
                    train_writer.add_summary(summary,i+ j*max_steps)
            sess.run(increment_epoch_ind)
        train_writer.close()
        test_writer.close()     
        print('\nTotal time:\t %4.3fs' % (time.time()-start_time))

### Complex nonlinear function
Example of learning a toy random nonlinear compelex-valued function. 
We separate real and imaginary parts and concatenate them into the inputs and outputs.

In [6]:
# Let's define a function to create a random linear operator, f, 
# and another function to generate (x, f(x)) pairs

def build_unknown_complex_nonlinear_func(m,n):
    # m is the input dimension
    # n is the output dimension
    
    #mat = tf.constant(np.random.rand(m,n),dtype=DTYPE)
    #def unknown_func(x):
    #   return tf.tensordot(x,mat,1) # check!
    
    mat0 = np.random.rand(m,n).astype(DTYPE_np) + 1j*np.random.rand(m,n).astype(DTYPE_np)
    mat1 = np.random.rand(m,n).astype(DTYPE_np) + 1j*np.random.rand(m,n).astype(DTYPE_np)
    
    def unknown_func(x):
        return np.matmul(np.power(x,2),mat0) + np.matmul(x,mat1)
    
    return [(mat0,mat1),unknown_func]

def generate_samples(m,batch_size,unknown_func):
    # Let's just assume that the input x values are random
    
    x = np.random.rand(batch_size,m)
    y = unknown_func(x) 
    
    # Concat real and imag
    x_cat = np.concatenate((np.real(x),np.imag(x)),axis=1)
    y_cat = np.concatenate((np.real(y),np.imag(y)),axis=1)
    
    return [x_cat,y_cat]

def train_complex_func():
    input_size = 8
    output_size = 13
    
    hidden_size = 30
    num_layers = 5
    batch_size = 25
    
    test_size = 200
    max_steps = 2001
    decay_steps = 500
    
    
    mat,unknown_func = build_unknown_complex_nonlinear_func(input_size,output_size)
    test_x, test_y = generate_samples(input_size,test_size,unknown_func)
    
    # Define the function that will be called to create a new set batch of training data.
    # It will be passed two arguments: epoch and global step (which is NOT reset at the start of each epoch)
    generate_training_samples_fun = lambda epoch_ind, global_step: generate_samples(input_size,batch_size,unknown_func) # 
    
    train_RBM_complete(input_size=2*input_size,output_size=2*output_size,hidden_size=hidden_size,num_layers = num_layers,unknown_func=unknown_func,
                  test_x=test_x, test_y = test_y, generate_training_samples_fun=generate_training_samples_fun,
                  batch_size=25, max_steps = max_steps, epochs =5, initial_learning_rate = 0.02, decay_rate = 1/math.e, decay_steps=decay_steps, prob_keep = 0.8)

In [7]:
reset_tf()

In [8]:
train_complex_func()

Epoch 0, Step 0000, MSE: 4.233e+01,  LearnRate: 2.000e-02, Time taken : 0.323s
Epoch 0, Step 0100, MSE: 3.496e-01,  LearnRate: 2.000e-02, Time taken : 2.049s
Epoch 0, Step 0200, MSE: 1.809e-01,  LearnRate: 2.000e-02, Time taken : 0.879s
Epoch 0, Step 0300, MSE: 1.076e-01,  LearnRate: 2.000e-02, Time taken : 0.802s
Epoch 0, Step 0400, MSE: 9.051e-02,  LearnRate: 2.000e-02, Time taken : 0.848s
Epoch 0, Step 0500, MSE: 9.097e-02,  LearnRate: 2.000e-02, Time taken : 0.990s
Epoch 0, Step 0600, MSE: 7.832e-02,  LearnRate: 7.358e-03, Time taken : 0.945s
Epoch 0, Step 0700, MSE: 7.033e-02,  LearnRate: 7.358e-03, Time taken : 0.878s
Epoch 0, Step 0800, MSE: 7.854e-02,  LearnRate: 7.358e-03, Time taken : 0.831s
Epoch 0, Step 0900, MSE: 7.081e-02,  LearnRate: 7.358e-03, Time taken : 0.871s
Epoch 0, Step 1000, MSE: 6.770e-02,  LearnRate: 7.358e-03, Time taken : 0.941s
Epoch 0, Step 1100, MSE: 5.948e-02,  LearnRate: 2.707e-03, Time taken : 0.974s
Epoch 0, Step 1200, MSE: 5.801e-02,  LearnRate: 2.70

In [38]:
import h5py
import numpy as np

In [39]:
fname = './ChrisFreierPhDCampaigns/campaign4a.h5'
f = h5py.File(fname,'r')

In [40]:
print("Keys: %s" % f.keys())

Keys: KeysView(<HDF5 file "campaign4a.h5" (mode r)>)


In [41]:
for key in f.keys():
    print(key)

(T,tau)
accelerometer
ai_kdown
ai_kup


In [42]:
accelerometer = np.array(f['accelerometer'])
ai_kdown = np.array(f['ai_kdown'])
ai_kup = np.array(f['ai_kup'])
T_tau = np.array(f['(T,tau)'])

In [44]:
print(accelerometer.shape)
print(ai_kdown.shape)
print(ai_kup.shape)

(100000001, 2)
(103427, 2)
(103403, 2)


## Theory

Consider a Mach-Zehnder interferometer with interrogation time $T$ and pulse duration $\tau$. The total interferometer phase is given by:
$$\Delta\Phi = \phi_{eff}^1 - 2\phi_{eff}^2 + \phi_{eff}^3 + \left(\mathrm{arg}(\Theta_0^1)-\mathrm{arg}(\Theta^3_0)\right)$$
where $\phi_{eff}^i$ describes the "light phase at the atomic positions during the three Raman pulses". $\Theta_0^i$ descibes secondary phase shifts due to "different light shifts between the first and last pulse".

Define the sensitivity function $g(t)$ as the effect on the total interferometer phase due to a phase jump $\delta\phi$ at time $t$. For the three-pulse Mach-Zehnder, with second pulse centered at $t=0$, $g(t)$ is given by:
$$g(t) = \begin{cases}
\sin(\Omega_rt), & 0<t\leq\tau \\
1, & \tau<t\leq T+\tau\\
-\sin(\Omega_r(T-t)), & T+\tau<t\leq T+2\tau \\
0, & t>T+2\tau
\end{cases}$$

For finite Raman pulse duration (i.e. $\tau>0$), the total interferometer phase is given by:
$$\Delta\Phi = \int_{-(T_2\tau)}^{(T+2\tau)}g(t)\frac{d\phi(t)}{dt}dt$$

To perform post-correction, we separtely calculate the phase offset caused by mirror vibrations:
$$\Phi_{vib} = k_{eff}\int_{t_1}^{t_3}g(t)v(t)dt$$
where $v(t)=\frac{1}{k_{eff}}\frac{d\phi}{dt}$ is the mirror velocity and $k_eff$ is the effective wavevector

## Data layout
Data is organised into campaigns, each containing ~10^5 interferometer runs. Use only 4a onwards.
Each `.h5` has three fields
- `accelerometer`: `[N_a,2]` linearly proportional to the raw out from the accelerometer as a time series. 1st column is linux time, 2nd column is signal
- `ai_kdown`: `[N_p,2]` total interferometer phase for kdown? configuration. 1st column is linux time, 2nd column is phase. Each row is the total interferometer phase for an Mach-Zehnder sequence with 2nd pulse centered at the given time
- `ai_kup`: `[N_p,2]` total interferometer phase for kup? configuration. Same layout as `ai_kdown`
- `(T,tau)`: `[2,]` the interferometer interrogation time `T` and the $\frac{\pi}{2}$ Raman pulse duration `tau`

First we need to prepare the data.

(1) Check timestamps of `ai_kdown`==`ai_kup`
- THEY DON'T; they aren't even necessarily the same length. Assume that each corresponds to a completely different subsequence of `accelerometer`
- Also, note that the distribution of delays between successive runs is weirdly distributed

(2) For each ai_* associate a contiguous subsequence of `accelerometer`
 - Ensure each subsequence is of equal length `N_s`
 - Will being left/right aligned w.r.t. rounding of `ai_kdown` timestamp to `accelerometer` timestamps affect anything?
 - Safest thing to do might be linearly interpolate `accelerometer` to ensure that `accelerometer` subsequence is correctly time-aligned with `ai_*`
 
(3) Construct our inputs and outputs
 - `x_input` = `[N_s,N_p]` array
 - `y_output` = `[N_p,2]` concatenate 2nd column of `ai_kdown` and `ai_kup`
    
## Check
Make sure we can reproduce original data (i.e. reproduce original $\Phi_{vib}$)
 - Do we have this data?
    
## Modelling
(1) *Model-free* Just use the inputs and labels as they are (assuming `N_s` is small enough)
 - Pros: easy
 - Cons: no ground-truth to compare against

(2) *Explicit transfer function* Apply unknown function in Laplace space ($\mathcal{L}\{y(t)\}(Z) =H(Z)\mathcal{L}\{x(t)\}(Z)$ )
 - Pros: ground-truth to compare against
 - Cons: less easy
    

In [63]:
list(f.keys())

['(T,tau)', 'accelerometer', 'ai_kdown', 'ai_kup']

In [62]:
list(f.items())

[('(T,tau)', <HDF5 dataset "(T,tau)": shape (2,), type "<f8">),
 ('accelerometer',
  <HDF5 dataset "accelerometer": shape (100000001, 2), type "<f8">),
 ('ai_kdown', <HDF5 dataset "ai_kdown": shape (103427, 2), type "<f8">),
 ('ai_kup', <HDF5 dataset "ai_kup": shape (103403, 2), type "<f8">)]

In [66]:
fname = './ChrisFreierPhDCampaigns/campaign5a.h5'
f0 = h5py.File(fname,'r')
list(f0.keys())
list(f0.items())

[('(T,tau)', <HDF5 dataset "(T,tau)": shape (2,), type "<f8">),
 ('accelerometer',
  <HDF5 dataset "accelerometer": shape (100000001, 2), type "<f8">),
 ('ai_kdown', <HDF5 dataset "ai_kdown": shape (80337, 2), type "<f8">),
 ('ai_kup', <HDF5 dataset "ai_kup": shape (80347, 2), type "<f8">)]

## Use Processed campaign4a
Matlab used to assign subsequences in `accelerometer` to each entry in `ai_kup` and `ai_kdown`.
Subsequence length is 195. ~100000 entries for each of `ai_kup` and `ai_kdown`.

In [21]:
fname = './campaign4a_proc.h5'
f0 = h5py.File(fname,'r')
list(f0.keys())
list(f0.items())

[('T', <HDF5 dataset "T": shape (1,), type "<f8">),
 ('ai_kdown', <HDF5 group "/ai_kdown" (3 members)>),
 ('ai_kup', <HDF5 group "/ai_kup" (3 members)>),
 ('tau', <HDF5 dataset "tau": shape (1,), type "<f8">)]

In [173]:
# Let's define a function that will generate indices for batches
# accounting for the possibility that the size of the dataset
# will not be an even multiple of the batch size
def generate_batches(N,batch_size=32):
    # N is the number of elements in the dataset
    if N<batch_size:
        raise ValueError('batch_size must be smaller than N')
    perm = np.random.permutation(N);
    if np.mod(N,batch_size)!=0:
        # We need to append to perm so that is is an even multiple of batch_size
        perm2= np.random.permutation(N);
        perm = np.concatenate((perm,perm2[0:batch_size-np.mod(N,batch_size)]))
    
    n_batches = np.int(len(perm)/batch_size)
    batches = perm.reshape(batch_size,n_batches)
    
    return batches

def get_xy_by_inds(x,y,inds):
    # Get values from 2Dx, 1Dy arrays
    x0 = x[inds,:]
    y0 = np.array(list(map(y.__getitem__,inds))) # because Python treats 1D arrays differently from ND arrays
    y0 = y0.reshape([len(y0),1])
    return x0,y0
    
def generate_testtrain(x,y,test_fraction,batch_size=32):
    # Assumes x.shape = [n_samples,input_size], y.shape = [input_size,]
    if test_fraction<0 or test_fraction>1:
        raise ValueError('test_fraction must be between 0 and 1')
    
    n_samples  = x.shape[0]
    test_size  = np.int(np.round(n_samples*test_fraction))
    if test_size<1:
        raise ValueError('test_fraction is too small')
    train_size = np.int(n_samples-test_size)
    
    # Permute x,y
    perm = np.random.permutation(n_samples)
    permi= np.argsort(perm)
    x0,y0= get_xy_by_inds(x,y,permi)
                    
    train_x = x0[:train_size,:]
    train_y = y0[:train_size,:]
    
    test_x  = x0[train_size:,:]
    test_y  = y0[train_size:,:]
    
    batches = generate_batches(train_size,batch_size)
    
    return test_x,test_y,train_x,train_y,batches

In [144]:
P = [0,5,4,6,7,6,9]
X = np.random.randint(10,size=[7,8])
p = np.random.permutation(len(P))
pi= np.argsort(p)
print(pi)
print(list(map(P.__getitem__,pi)))
print(X)
print(X[pi,:])
test_x,test_y,train_x,train_y,batches = generate_testtrain(X,P,0.25,batch_size = 3)

[4 2 1 3 5 0 6]
[7, 4, 5, 6, 6, 0, 9]
[[8 0 7 7 3 6 7 6]
 [3 6 1 9 3 4 7 5]
 [1 3 8 6 4 9 5 0]
 [5 5 7 8 0 0 2 4]
 [2 4 1 2 5 2 0 4]
 [3 3 1 7 0 8 3 3]
 [7 4 6 2 6 9 5 6]]
[[2 4 1 2 5 2 0 4]
 [1 3 8 6 4 9 5 0]
 [3 6 1 9 3 4 7 5]
 [5 5 7 8 0 0 2 4]
 [3 3 1 7 0 8 3 3]
 [8 0 7 7 3 6 7 6]
 [7 4 6 2 6 9 5 6]]


In [146]:
get_xy_by_inds(X,P,batches[:,1])

(array([[5, 5, 7, 8, 0, 0, 2, 4],
        [3, 6, 1, 9, 3, 4, 7, 5],
        [1, 3, 8, 6, 4, 9, 5, 0]]), array([6, 5, 4]))

In [81]:
print(list(dset.keys()))
print(repr(dset['acc']))
print(repr(dset['phase']))
dset['phase'].shape[0]

['acc', 'phase', 'timestamp']
<HDF5 dataset "acc": shape (103403, 195), type "<f8">
<HDF5 dataset "phase": shape (103403,), type "<f8">


103403

In [233]:
def func_deep_learner_arbshape(x,m,n,h,is_test,global_step,prob_keep,act_func=tf.nn.relu):
    # x is the input variable, probably a TensorFlow placeholder object
    # m is the input dimension
    # n is the output dimension
    # h is a LIST of hidden neurons
    # act_func is the activation function to be applied
    # is_test is a flag is use to control batch normalisation behaviour during inferene on test data
    # iteration is the iteration counter used inside the training loop; required for batch normalisation
    
    num_layers = len(h)
    
    bn_moving_avg_updates = []

    with tf.variable_scope('func_learner'):
        # 0th hidden layer
        hidden_layer, update_moving_avg = get_layer_complete(x,m,h[0],'hidden_layer_0',is_test,prob_keep,global_step)
        bn_moving_avg_updates.append(update_moving_avg)

        # Other hidden layers
        for i in range(num_layers-1):
            hidden_layer, update_moving_avg = get_layer_complete(hidden_layer,h[i],h[i+1],'hidden_layer_'+str(i+1),is_test,prob_keep,global_step)
            bn_moving_avg_updates.append(update_moving_avg)

        # Output layer
        weights1= tf.get_variable(name='output_layer_weights',
                              shape=[h[-1],n],
                              initializer=tf.random_normal_initializer(),
                              dtype=DTYPE)

    return tf.matmul(hidden_layer,weights1), bn_moving_avg_updates

In [226]:
def train_RBM_complete_mccv(input_size=1,output_size=1,hidden_sizes=[5,5],x_data=None,y_data=None,
                  batch_size=32, epochs =5, test_fraction=0.1, test_interval=250,
                  initial_learning_rate = 0.02, decay_rate = 1/math.e, decay_steps=2000,
                  epoch_learning_rate_decay = 0.5, epoch_learning_rate_interval = 10,
                  prob_keep = 0.8):
    # Using montecarlo crossvalidation
    
    with tf.Session() as sess:
        
        # Generate test data initially for convenience
        test_x,test_y,train_x_full,train_y_full,batches = generate_testtrain(x_data,y_data,test_fraction,batch_size)
        max_steps = batches.shape[1]
            
        with tf.name_scope('input'):
            # Placeholder variables which will be fed data at train time
            x  = tf.placeholder(DTYPE,[None,input_size],name='x_input')
            y  = tf.placeholder(DTYPE,[None,output_size],name='y_input')
        
        with tf.name_scope('control_inputs'):
            global_step = tf.Variable(0, trainable=False,name='global_step')
            epoch_ind = tf.Variable(0,trainable=False,name='epoch_ind')
            increment_epoch_ind = tf.assign_add(epoch_ind,1,name='increment_epoch_ind')
            is_test = tf.contrib.eager.Variable(False, trainable=False,name='is_test')

        # Predictions of the NN
        y_pred, bn_moving_avg_updates = func_deep_learner_arbshape(x,input_size,output_size,hidden_sizes,is_test,global_step,1)

        with tf.name_scope('mean_squared_error'):
            mse = tf.losses.mean_squared_error(y,y_pred)
        tf.summary.scalar('mean_squared_error',mse)

        with tf.name_scope('train'):
            global_step_in_epoch = global_step - epoch_ind*max_steps
            #learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step_in_epoch, decay_steps, decay_rate, staircase=True)
            learning_rate = tf.train.exponential_decay(initial_learning_rate, epoch_ind, epoch_learning_rate_interval, epoch_learning_rate_decay, staircase=True)
            tf.summary.scalar('learning_rate', learning_rate)
            train_step = tf.train.AdamOptimizer(learning_rate).minimize(mse, global_step = global_step)

        # Create merged summary object and file writers
        merged_summaries = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter(log_dir + '/train', sess.graph)
        test_writer = tf.summary.FileWriter(log_dir + '/test', sess.graph)
        
        # Initialise variables
        sess.run(tf.global_variables_initializer())
        
        # Train the model
        start_time = time.time()
        old_time = start_time
        for j in range(epochs):
            # At each epoch, regenerate test data, train data, and batches
            test_x,test_y,train_x_full,train_y_full,batches = generate_testtrain(x_data,y_data,test_fraction,batch_size)
            n_batches = batches.shape[1]
                
            print('Starting Epoch %d, with %d batches' % (j,max_steps))
            for i in range(n_batches):
                if i % test_interval ==0:
                    # Check how our model performs against the test data
                    [summary, mse_val] = sess.run([merged_summaries,mse], feed_dict={x: test_x, y: test_y, is_test: True})
                    test_writer.add_summary(summary,i + j*max_steps)
                    curr_time = time.time()
                    print('\tEpoch %d, Step %04d, MSE: %4.3e,  LearnRate: %4.3e, Time taken : %4.3fs' % (sess.run(epoch_ind),i, mse_val, sess.run(learning_rate),curr_time-old_time))
                    old_time = curr_time
                else:
                    # Generate new sample data and train our model
                    train_x, train_y = get_xy_by_inds(train_x_full,train_y_full,batches[:,i])
                    summary, _ = sess.run([merged_summaries, train_step], feed_dict={x: train_x, y: train_y, is_test: False})
                    sess.run(bn_moving_avg_updates, feed_dict={x: train_x, y: train_y, is_test: False})
                    train_writer.add_summary(summary,i+ j*max_steps)
            sess.run(increment_epoch_ind)
        train_writer.close()
        test_writer.close()     
        print('\nTotal time:\t %4.3fs' % (time.time()-start_time))

In [229]:
def train_model_ai_kup(acc,phase):
    n_samples  = acc.shape[0]
    seq_length = acc.shape[1]
    
    input_size  = seq_length
    output_size = 1
    
    hidden_sizes = [80,40,20,10,5]
    num_layers  = 5
    batch_size  = 32
    
    mccv_steps   = 100
    test_fraction= 0.1
    
    train_RBM_complete_mccv(input_size=input_size,output_size=output_size,hidden_sizes=hidden_sizes,
                            x_data=acc,y_data=phase,batch_size=batch_size, epochs=mccv_steps,test_fraction=test_fraction,test_interval=100,
                            initial_learning_rate = 0.5, decay_rate = 1/math.e, decay_steps=3000,
                            epoch_learning_rate_decay = 0.5, epoch_learning_rate_interval = 10,
                            prob_keep = 0.8)

In [161]:
print(list(dset.keys()))
print(repr(dset['acc']))
print(repr(dset['phase']))
dset['phase'].shape[0]

['acc', 'phase', 'timestamp']
<HDF5 dataset "acc": shape (103403, 195), type "<f8">
<HDF5 dataset "phase": shape (103403,), type "<f8">


103403

In [156]:
ai_kup_phase = np.array(list(dset['phase']))
ai_kup_acc = np.array(list(dset['acc']))

In [234]:
reset_tf()

In [235]:
train_model_ai_kup(ai_kup_acc,ai_kup_phase)

Starting Epoch 0, with 2909 batches
	Epoch 0, Step 0000, MSE: 1.143e+14,  LearnRate: 5.000e-01, Time taken : 0.501s
	Epoch 0, Step 0100, MSE: 1.137e+14,  LearnRate: 5.000e-01, Time taken : 2.063s
	Epoch 0, Step 0200, MSE: 1.115e+14,  LearnRate: 5.000e-01, Time taken : 0.972s
	Epoch 0, Step 0300, MSE: 1.077e+14,  LearnRate: 5.000e-01, Time taken : 0.975s
	Epoch 0, Step 0400, MSE: 1.024e+14,  LearnRate: 5.000e-01, Time taken : 0.970s
	Epoch 0, Step 0500, MSE: 9.581e+13,  LearnRate: 5.000e-01, Time taken : 0.912s
	Epoch 0, Step 0600, MSE: 8.849e+13,  LearnRate: 5.000e-01, Time taken : 1.087s
	Epoch 0, Step 0700, MSE: 8.057e+13,  LearnRate: 5.000e-01, Time taken : 1.058s
	Epoch 0, Step 0800, MSE: 7.246e+13,  LearnRate: 5.000e-01, Time taken : 1.090s
	Epoch 0, Step 0900, MSE: 6.418e+13,  LearnRate: 5.000e-01, Time taken : 0.993s
	Epoch 0, Step 1000, MSE: 5.589e+13,  LearnRate: 5.000e-01, Time taken : 0.961s
	Epoch 0, Step 1100, MSE: 4.868e+13,  LearnRate: 5.000e-01, Time taken : 1.095s
	Epo